In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
# from tqdm import tqdm_notebook as tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loades
from src.trainer import *

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'create_data_loades'

# TODO:
* Переместить куда-нибудь окончательную обработку датасета
* Переместить деление на трейн-тест
* Зафиксировать сиды и длинны последовательностей в параметрах
* Написать скрипты для обучения
* История по эпохам

In [2]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(r'C:\Users\admin\Google Drive\Datasets\CERT_output')
answers_dir = Path(r"C:\Users\admin\Google Drive\Datasets\CERT\answers")

run_name = 'test'

main_answers_file = answers_dir / "insiders.csv"

In [5]:
actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl', main_answers_file, min_length=50, max_length=200)

C:\Anaconda3\envs\python3.6-torch\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


# Train

TODO:
* Эксперименты с LR Scheduling

In [6]:
cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0)

params = get_params()

device = 'cuda'

NameError: name 'create_data_loaders' is not defined

In [ ]:
lstm_encoder = LSTM_Encoder(params['lstm_encoder'])
criterion = nn.BCELoss()
optimizer = optim.Adam(lstm_encoder.parameters())

train_engine = create_supervised_trainer_lstm(
                                        lstm_encoder, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch_lstm,
                                        log_dir=(output_dir / 'logs' / run_name).as_posix(),
                                        checkpoint_dir=output_dir / 'checkpoints' / run_name
                                       )

# val_engine = create_supervised_evaluator(
#         lstm_encoder, metrics={"loss": Loss(criterion, batch_size=lambda x: x.shape[0])}, device=device
# )

# @train_engine.on(Events.EPOCH_COMPLETED)
# def log_training_results(trainer):
#     val_engine.run(train_batcher)
#     metrics = evaluator.state.metrics
#     print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
#           .format(trainer.state.epoch, metrics['accuracy']))

# @train_engine.on(Events.EPOCH_COMPLETED)
# def log_validation_results(trainer):
#     val_engine.run(val_batcher)
#     metrics = evaluator.state.metrics
#     print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
#           .format(trainer.state.epoch, metrics['accuracy']))

train_engine.run(train_loader, max_epochs=10)

In [ ]:
Cert